### Heroes Of Pymoli Data Analysis
* Of the 1163 active players, the vast majority are male (84%). There also exists, a smaller, but notable proportion of female players (14%).

* Our peak age demographic falls between 20-24 (44.8%) with secondary groups falling between 15-19 (18.60%) and 25-29 (13.4%).  
-----

### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# import pandas and os
import pandas as pd
import os

# define file path to open purchase data
file_purchase_data = os.path.join("Resources", "purchase_data.csv")

# Read purchase data file and store into Pandas data frame
purchase_data_df = pd.read_csv(file_purchase_data)
purchase_data_df

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56
2,2,Ithergue48,24,Male,92,Final Critic,4.88
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27
4,4,Iskosia90,23,Male,131,Fury,1.44
...,...,...,...,...,...,...,...
775,775,Aethedru70,21,Female,60,Wolf,3.54
776,776,Iral74,21,Male,164,Exiled Doomblade,1.63
777,777,Yathecal72,20,Male,67,"Celeste, Incarnation of the Corrupted",3.46
778,778,Sisur91,7,Male,92,Final Critic,4.19


## Player Count

* Display the total number of players


In [2]:
player_count = len(purchase_data_df["SN"].unique())
total_players_df = pd.DataFrame({"Total Players": [player_count]})
total_players_df

,Total Players
0,576


## Purchasing Analysis (Total)

* Run basic calculations to obtain number of unique items, average price, etc.


* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame


In [3]:
item_count = len(purchase_data_df["Item Name"].unique())
average_purchase_price = round(purchase_data_df["Price"].mean(),2)
number_of_purchases = len(purchase_data_df["Purchase ID"])
total_revenue = purchase_data_df["Price"].sum()
purchasing_anaysis_df = pd.DataFrame({"Number of Unique Items": [item_count],
                                      "Average Price": [average_purchase_price],
                                      "Number of Purchases": [number_of_purchases],
                                      "Total Revenue": [total_revenue]
                                     })

purchasing_anaysis_df

,Number of Unique Items,Average Price,Number of Purchases,Total Revenue
0,179,3.05,780,2379.77


## Gender Demographics

* Percentage and Count of Male Players


* Percentage and Count of Female Players


* Percentage and Count of Other / Non-Disclosed




In [4]:
player_data_df = purchase_data_df.loc[:,["SN", "Age", "Gender"]]
player_data_df.sort_values("SN", inplace = True)
player_data_df.drop_duplicates(subset = None, keep = 'first', inplace = True)
total_players = len(player_data_df)
gender_df = player_data_df.set_index("Gender")

female_players = gender_df.loc["Female", "SN"]
male_players = gender_df.loc["Male", "SN"]
other_players = gender_df.loc["Other / Non-Disclosed", "SN"]

demograph_gend_df = pd.DataFrame({"Gender": ["Male", "Female", "Other / Non-Disclosed"],   
                             "Total Count": [len(male_players), len(female_players), len(other_players)],
                             "Percentage of Players": [len(male_players) / total_players, len(female_players) / total_players, len(other_players) / total_players]
                            })

demograph_gend_df = demograph_gend_df.set_index("Gender")
demograph_gend_df


,Total Count,Percentage of Players
Gender,,
Male,484,0.840278
Female,81,0.140625
Other / Non-Disclosed,11,0.019097



## Purchasing Analysis (Gender)

* Run basic calculations to obtain purchase count, avg. purchase price, avg. purchase total per person etc. by gender




* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame

In [5]:
purch_by_male_df = purchase_data_df.loc[purchase_data_df["Gender"] == "Male",:]
purch_by_female_df = purchase_data_df.loc[purchase_data_df["Gender"] == "Female",:]
purch_by_other_df = purchase_data_df.loc[purchase_data_df["Gender"] == "Other / Non-Disclosed",:]
group_by_male_df = purch_by_male_df.groupby(["SN"])
group_by_female_df = purch_by_female_df.groupby(["SN"])
group_by_other_df = purch_by_other_df.groupby(["SN"])
total_by_male = group_by_male_df["Price"].sum()
total_by_female = group_by_female_df["Price"].sum()
total_by_other = group_by_other_df["Price"].sum()

purch_analysis_gend_df = pd.DataFrame({"Gender": ["Male", "Female", "Other / Non-Disclosed"],
                                       "Purchase Count": [len(purch_by_male_df), len(purch_by_female_df), len(purch_by_other_df)],
                                       "Average Purchase Price": [round(purch_by_male_df["Price"].sum() / len(purch_by_male_df),2), round(purch_by_female_df["Price"].sum() / len(purch_by_female_df),2),
                                                                  round(purch_by_other_df["Price"].sum() / len(purch_by_other_df),2)], 
                                       "Total Purchase Value": [purch_by_male_df["Price"].sum(), purch_by_female_df["Price"].sum(), purch_by_other_df["Price"].sum()],
                                       "Avg Total Purchase per Person": [round(total_by_male.sum() / len(total_by_male),2), round(total_by_female.sum() / len(total_by_female),2),
                                                                         round(total_by_other.sum() / len(total_by_other),2)]
                                      })

purch_analysis_gend_df


,Gender,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
0,Male,652,3.02,1967.64,4.07
1,Female,113,3.20,361.94,4.47
2,Other / Non-Disclosed,15,3.35,50.19,4.56


## Age Demographics

* Establish bins for ages


* Categorize the existing players using the age bins. Hint: use pd.cut()


* Calculate the numbers and percentages by age group


* Create a summary data frame to hold the results


* Optional: round the percentage column to two decimal points


* Display Age Demographics Table


In [6]:
age_bin = ["<10", "10-14", "15-19", "20-24", "25-29", "30-34", "35-39", "40+", "45+"]
age_list = purchase_data_df["Age"]
age_group = pd.cut(age_list,range(5,55,5), right=False, labels=age_bin)

purchase_data_with_age_group_df = purchase_data_df
purchase_data_with_age_group_df["Age Group"] = age_group
player_data_with_age_group_df = purchase_data_with_age_group_df.loc[:,["SN", "Age", "Gender", "Age Group"]]
player_data_with_age_group_df.sort_values("SN", inplace = True)
player_data_with_age_group_df.drop_duplicates(subset = None, keep = 'first', inplace = True)
player_data_with_age_group_df["Age Group"] = player_data_with_age_group_df["Age Group"].replace({"45+": "40+"})
age_df = player_data_with_age_group_df.set_index("Age Group")

under10 = age_df.loc["<10","SN"]
tento14 = age_df.loc["10-14","SN"]
fifteento19 = age_df.loc["15-19","SN"]
twentyto24 = age_df.loc["20-24","SN"]
twenty5to29 = age_df.loc["25-29","SN"]
thirtyto34 = age_df.loc["30-34","SN"]
thirty5to39 = age_df.loc["35-39","SN"]
fourtyplus = age_df.loc["40+","SN"]

demograph_age_df = pd.DataFrame({"Age Group": ["<10", "10-14", "15-19", "20-24", "25-29", "30-34", "35-39", "40+"],
                                "Total Count": [len(under10), len(tento14), len(fifteento19), len(twentyto24), len(twenty5to29), len(thirtyto34), len(thirty5to39), len(fourtyplus)],
                                 "Percentage of Players": [len(under10)/total_players, len(tento14)/total_players, len(fifteento19)/total_players, len(twentyto24)/total_players, 
                                                           len(twenty5to29)/total_players, len(thirtyto34)/total_players, len(thirty5to39)/total_players, len(fourtyplus)/total_players]
                                })

demograph_age_df

,Age Group,Total Count,Percentage of Players
0,<10,17,0.029514
1,10-14,22,0.038194
2,15-19,107,0.185764
3,20-24,258,0.447917
4,25-29,77,0.133681
5,30-34,52,0.090278
6,35-39,31,0.053819
7,40+,12,0.020833


## Purchasing Analysis (Age)

* Bin the purchase_data data frame by age


* Run basic calculations to obtain purchase count, avg. purchase price, avg. purchase total per person etc. in the table below


* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame

In [15]:
purchase_data_with_age_group_df["Age Group"] = purchase_data_with_age_group_df["Age Group"].replace({"45+": "40+"})

purch_by_under10 = purchase_data_with_age_group_df.loc[purchase_data_with_age_group_df["Age Group"] == "<10",:]
purch_by_tento14 = purchase_data_with_age_group_df.loc[purchase_data_with_age_group_df["Age Group"] == "10-14",:]
purch_by_fifteento19 = purchase_data_with_age_group_df.loc[purchase_data_with_age_group_df["Age Group"] == "15-19",:]
purch_by_twentyto24 = purchase_data_with_age_group_df.loc[purchase_data_with_age_group_df["Age Group"] == "20-24",:]
purch_by_twenty5to29 = purchase_data_with_age_group_df.loc[purchase_data_with_age_group_df["Age Group"] == "25-29",:]
purch_by_thirtyto34 = purchase_data_with_age_group_df.loc[purchase_data_with_age_group_df["Age Group"] == "30-34",:]
purch_by_thirty5to39 = purchase_data_with_age_group_df.loc[purchase_data_with_age_group_df["Age Group"] == "35-39",:]
purch_by_fourtyplus = purchase_data_with_age_group_df.loc[purchase_data_with_age_group_df["Age Group"] == "40+",:]

group_by_under10 = purch_by_under10.groupby(["SN"])
group_by_tento14 = purch_by_tento14.groupby(["SN"])
group_by_fifteento19 = purch_by_fifteento19.groupby(["SN"])
group_by_twentyto24 = purch_by_twentyto24.groupby(["SN"])
group_by_twenty5to29 = purch_by_twenty5to29.groupby(["SN"])
group_by_thirtyto34 = purch_by_thirtyto34.groupby(["SN"])
group_by_thirty5to39 = purch_by_thirty5to39.groupby(["SN"])
group_by_fourtyplus = purch_by_fourtyplus.groupby(["SN"])

total_by_under10 = group_by_under10["Price"].sum()
total_by_tento14 = group_by_tento14["Price"].sum()
total_by_fifteento19 = group_by_fifteento19["Price"].sum()
total_by_twentyto24 = group_by_twentyto24["Price"].sum()
total_by_twenty5to29 = group_by_twenty5to29["Price"].sum()
total_by_thirtyto34 = group_by_thirtyto34["Price"].sum()
total_by_thirty5to39 = group_by_thirty5to39["Price"].sum()
total_by_fourtyplus = group_by_fourtyplus["Price"].sum()

purch_analysis_age_df = pd.DataFrame({"Age Group": ["<10", "10-14", "15-19", "20-24", "25-29", "30-34", "35-39", "40+"],
                                       "Purchase Count": [len(purch_by_under10), len(purch_by_tento14), len(purch_by_fifteento19), len(purch_by_twentyto24),
                                                        len(purch_by_twenty5to29),len(purch_by_thirtyto34),len(purch_by_thirty5to39),len(purch_by_fourtyplus)],
                                       "Average Purchase Price": [round(purch_by_under10["Price"].sum()/len(purch_by_under10),2),
                                                                  round(purch_by_tento14["Price"].sum()/len(purch_by_tento14),2),
                                                                  round(purch_by_fifteento19["Price"].sum()/len(purch_by_fifteento19),2), 
                                                                  round(purch_by_twentyto24["Price"].sum()/len(purch_by_twentyto24),2), 
                                                                  round(purch_by_twenty5to29["Price"].sum()/len(purch_by_twenty5to29),2), 
                                                                  round(purch_by_thirtyto34["Price"].sum()/len(purch_by_thirtyto34),2), 
                                                                  round(purch_by_thirty5to39["Price"].sum()/len(purch_by_thirty5to39),2), 
                                                                  round(purch_by_fourtyplus["Price"].sum()/len(purch_by_fourtyplus),2)],
                                       "Total Purchase Value": [purch_by_under10["Price"].sum(), purch_by_tento14["Price"].sum(), purch_by_fifteento19["Price"].sum(), 
                                                                purch_by_twentyto24["Price"].sum(), purch_by_twenty5to29["Price"].sum(), purch_by_thirtyto34["Price"].sum(),
                                                                purch_by_thirty5to39["Price"].sum(), purch_by_fourtyplus["Price"].sum()],
                                       "Avg Total Purchase per Person": [round(total_by_under10.sum()/len(total_by_under10),2),
                                                                         round(total_by_tento14.sum()/len(total_by_tento14),2),
                                                                         round(total_by_fifteento19.sum()/len(total_by_fifteento19),2), 
                                                                         round(total_by_twentyto24.sum()/len(total_by_twentyto24),2), 
                                                                         round(total_by_twenty5to29.sum()/len(total_by_twenty5to29),2), 
                                                                         round(total_by_thirtyto34.sum()/len(total_by_thirtyto34),2), 
                                                                         round(total_by_thirty5to39.sum()/len(total_by_thirty5to39),2), 
                                                                         round(total_by_fourtyplus.sum()/len(total_by_fourtyplus),2)]
                                     })

purch_analysis_age_df



,Age Group,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
0,<10,23,3.35,77.13,4.54
1,10-14,28,2.96,82.78,3.76
2,15-19,136,3.04,412.89,3.86
3,20-24,365,3.05,1114.06,4.32
4,25-29,101,2.90,293.00,3.81
5,30-34,73,2.93,214.00,4.12
6,35-39,41,3.60,147.67,4.76
7,40+,13,2.94,38.24,3.19


## Top Spenders

* Run basic calculations to obtain the results in the table below


* Create a summary data frame to hold the results


* Sort the total purchase value column in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the summary data frame



## Most Popular Items

* Retrieve the Item ID, Item Name, and Item Price columns


* Group by Item ID and Item Name. Perform calculations to obtain purchase count, item price, and total purchase value


* Create a summary data frame to hold the results


* Sort the purchase count column in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the summary data frame



## Most Profitable Items

* Sort the above table by total purchase value in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the data frame

